In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px



# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error


from utils import cyclical_encoder
from utils import train_val_test_split
from utils import train_val_test_split_np
from utils import get_model_metrics

import seaborn as sns

#xgboost
from xgboost import XGBRegressor as xgbr
from xgboost import XGBRFRegressor as xgbrfr

In [2]:
def load_data(resolution, nan_value, val_days, test_days):
    
    if resolution == '16x10':
        path = '/home/tjarke/Desktop/Residual_Load/work/Feature_channels/'
        file = f'feature_channel_{resolution}.npy'
        feature_channel = np.load(path+file, allow_pickle=False)

        # get rid of all nans - replace them with nan_value
        feature_channel = np.nan_to_num(feature_channel, nan=nan_value)
        
        # rearange the axis to get the examples as the first dimension for the use in tensorflow
        feature_channel = np.moveaxis(feature_channel, -1, 0)
        feature_channel_load = feature_channel[:,:,:,:7]
        print(f'The shape of the feature_channel_load is {feature_channel_load.shape}')
        feature_channel_gen = feature_channel       
        print(f'The shape of the feature_channel_gen is {feature_channel_gen.shape}')
        
    else:
        path = '/home/tjarke/Desktop/Residual_Load/work/Feature_channels/'
        weather_file = f'feature_channel_{resolution}.npy'
        feature_channel_load = np.load(path+weather_file, allow_pickle=False)
        feature_channel_load = np.nan_to_num(feature_channel_load, nan=nan_value)        
        feature_channel_load = np.moveaxis(feature_channel_load, -1, 0)
        print(f'The shape of the feature_channel_load is {feature_channel_load.shape}')
        
        path = '~/Desktop/Residual_Load/work/Feature_channels/'
        ic_file = f'installed_capacities_{resolution}.npy'
        ic_channel = np.load(path+ic_file, allow_pickle=False)
        ic_channel = np.moveaxis(ic_channel, -1, 0)
        feature_channel_gen = np.dstack((feature_channel_load, ic_channel))
        print(f'The shape of the feature_channel_gen is {feature_channel_gen.shape}')

        
    # load the holidays in germany
    path = '/home/tjarke/Desktop/Residual_Load/work/'
    hol_file = 'holidays_encoded.csv'
    df_hol = pd.read_csv(path+hol_file,parse_dates=[0])
    print(f'The df_hol has the shape {df_hol.shape}')
        
    # load the realised values
    path = '~/Desktop/Residual_Load/work/'
    file_realised = 'Day_ahead_dataset.csv'
    df_realised = pd.read_csv(path+file_realised,parse_dates=[1])
    df_realised.drop(columns=["index"],inplace=True)
    # fill all solar nan with zero (even though some times it is during the day)
    df_realised.loc[df_realised['Realised/Solar in MAW'].isnull(),'Realised/Solar in MAW']  = 0
    # fill all other nans with interpolat
    df_realised = df_realised.interpolate(method='linear')
    
#     df_entsoe = df_realised.loc[:,['Day ahead/System total load in MAW', 'Day ahead/Solar in MAW',
#        'Day ahead/Wind Onshore in MAW', 'Day ahead/Wind Offshore in MAW']].copy()
    
#     df_realised.drop(columns=['index', 'Day ahead/System total load in MAW', 'Day ahead/Solar in MAW',
#        'Day ahead/Wind Onshore in MAW', 'Day ahead/Wind Offshore in MAW'], inplace=True)
    # get rid of nans
#     df_realised = df_realised.interpolate(method='linear')
    print(f'The shape of the df_realised is {df_realised.shape}')

    
    target_vars = ["Realised/System total load in MAW",
                   "Realised/Wind Offshore in MAW",
                   "Realised/Wind Onshore in MAW",
                   "Realised/Solar in MAW"]
    
    # Normalize the data
#     X_max = 100_000
#     for col in target_vars:
#         df_realised[col] /= X_max
    
    # create time columns
    
    df_realised['Year'] = df_realised['Date'].dt.year-2014    
    df = cyclical_encoder(df_realised,1440,"minute")
    df = cyclical_encoder(df,12,"month")
    df = cyclical_encoder(df,7,"weekday")

        
    # kill the last 100 rows since these are nan values
    # skip the first year
#     n_start = 0
    n_start = int(96*365*0)
    num_rows = -100
    df = df.iloc[n_start:num_rows,:]
#     df_entsoe = df_entsoe.iloc[:num_rows,:]
    df_hol = df_hol.iloc[n_start:-8,:]
    feature_channel_gen = feature_channel_gen[n_start:num_rows,:,:,:]
    feature_channel_load = feature_channel_load[n_start:num_rows,:,:,:]
    print(f'After getting rid of the last 100 rows and startin after {n_start} steps the shapes are:')
    print(f'df_realised {df.shape}, df_hol {df_hol.shape}, fc_gen {feature_channel_gen.shape}') 
    print(f'fc_load {feature_channel_load.shape}')
    
    # Split the data into train, val, test
    X_train_day, X_train_realised, \
    X_val_day, X_val_realised, \
    X_test_day, X_test_realised = train_val_test_split(df, target_vars, val_days, test_days)
    
    y_true = X_test_realised.copy()
    X_test_realised.drop(columns='Date', inplace=True)
    
    print(f'\nThe order of the columns for X_realised is: {X_train_realised.columns}')
#     print(f'The order of the columns for X_realised is: {X_val_realised.columns}')
#     print(f'The order of the columns for X_realised is: {X_test_realised.columns}\n')
    
    print(f'\nThe order of the columns for X_day is: {X_train_day.columns}')
#     print(f'The order of the columns for X_realised is: {X_val_realised.columns}')
#     print(f'The order of the columns for X_realised is: {X_test_realised.columns}\n')
    
    y_train = X_train_realised.to_numpy(copy=True)
    y_val = X_val_realised.to_numpy(copy=True)
    X_train_day = X_train_day.to_numpy(copy=True)
    X_train_realised = X_train_realised.to_numpy(copy=True)
    X_val_day = X_val_day.to_numpy(copy=True)
    X_val_realised = X_val_realised.to_numpy(copy=True)
    X_test_day = X_test_day.to_numpy(copy=True)
    X_test_realised = X_test_realised.to_numpy(copy=True)
    
    # split the feature channels into train, val, test
    X_train_fc_gen, X_val_fc_gen, X_test_fc_gen = train_val_test_split_np(feature_channel_gen, 
                                                                           val_days, test_days)

    X_train_fc_load, X_val_fc_load, X_test_fc_load = train_val_test_split_np(feature_channel_load, 
                                                                              val_days, test_days)
    
    # pre-process the holidays and split it into train, val and test
    df_hol['holidays'] = df_hol.mean(axis=1)
    df_hol.rename(columns={'time':'Date'},inplace=True)
    cols = [col for col in df_hol.columns if col not in ['holidays']]
#     df_hol['Date'] = 0 # so that the train, val, test split works  
    X_train_hol, _, X_val_hol, _, X_test_hol, _ = train_val_test_split(df_hol, cols, val_days, test_days)
    
    X_train_hol = X_train_hol.to_numpy(copy=True)
    X_val_hol = X_val_hol.to_numpy(copy=True)
    X_testn_hol = X_test_hol.to_numpy(copy=True)
    
    y_test = y_true.iloc[:, :-1].to_numpy(copy=True)
    
    return y_train, y_val, y_true, y_test,\
           X_train_realised, X_val_realised, X_test_realised, \
           X_train_day, X_val_day, X_test_day, \
           X_train_hol, X_val_hol, X_test_hol, \
           X_train_fc_gen, X_val_fc_gen, X_test_fc_gen, \
           X_train_fc_load, X_val_fc_load, X_test_fc_load 

In [3]:
y_train, y_val, y_true, y_test,\
X_train_realised, X_val_realised, X_test_realised, \
X_train_day, X_val_day, X_test_day, \
X_train_hol, X_val_hol, X_test_hol, \
X_train_fc_gen, X_val_fc_gen, X_test_fc_gen, \
X_train_fc_load, X_val_fc_load, X_test_fc_load = load_data('16x10', 0, 90, 90)

The shape of the feature_channel_load is (204096, 16, 10, 7)
The shape of the feature_channel_gen is (204096, 16, 10, 9)
The df_hol has the shape (204004, 17)
The shape of the df_realised is (204096, 9)
After getting rid of the last 100 rows and startin after 0 steps the shapes are:
df_realised (203996, 16), df_hol (203996, 17), fc_gen (203996, 16, 10, 9)
fc_load (203996, 16, 10, 7)
The shape of the data set is: (203996, 16)
--------------------------------------------
The shape of the train set is: (186716, 11)
The shape of the target variable is: (186716, 4)
--------------------------------------------
--------------------------------------------
The shape of the validation set is: (8640, 11)
The shape of the target variable for the validation set is: (8640, 4)
--------------------------------------------

--------------------------------------------
The shape of the test set is: (8640, 11)
The shape of the target variable for the test set is: (8640, 5)
------------------------------

In [4]:
print(f'the y_sets shape: {y_train.shape}, {y_val.shape}, {y_true.shape}')
print(f'the realised sets shape: {X_train_realised.shape}, {X_val_realised.shape}, {X_test_realised.shape}')
print(f'the day set shape: {X_train_day.shape}, {X_val_day.shape}, {X_test_day.shape}')
print(f'the fc_gen shape: {X_train_fc_gen.shape}, {X_val_fc_gen.shape}, {X_test_fc_gen.shape}')
print(f'the fc_load shape: {X_train_fc_load.shape}, {X_val_fc_load.shape}, {X_test_fc_load.shape}')

the y_sets shape: (186716, 4), (8640, 4), (8640, 5)
the realised sets shape: (186716, 4), (8640, 4), (8640, 4)
the day set shape: (186716, 11), (8640, 11), (8640, 11)
the fc_gen shape: (186716, 16, 10, 9), (8640, 16, 10, 9), (8640, 16, 10, 9)
the fc_load shape: (186716, 16, 10, 7), (8640, 16, 10, 7), (8640, 16, 10, 7)


In [5]:
# create a better generator function:

def generator(recognizer, 
              time_steps_weather, time_steps_realised, time_steps_days,
              feature_matrix_fc_gen, feature_matrix_fc_load, 
              feature_matrix_day, feature_matrix_hol,
              feature_matrix_realised, target_matrix):
    '''
    INPUT:
    recognizer: A string: either 'train', 'val' or 'test'. Make sure each variable gets a unique name
    time_steps_weather: A list: list of time steps to look in the past for the weather data/feature_channel, 
    e.g. 4 means get the value from one hour ago
    time_steps_realised: same as before, but for the realised/actual values
    feature_matrix_fc: numpy array with the information for the feature_channel/weather data
    feature_matrix_lag: same as before, but for the realised/actual data
    target_matrix: numpy array witht the target data
    '''
    max_steps_weather = max(time_steps_weather)
    max_steps_realised = max(time_steps_realised)
    max_steps_days = max(time_steps_days)
    
    # create dictionaries of lists for the features and a list for the target:
    steps_weather_gen_dict = dict()
    steps_weather_load_dict = dict()
    steps_realised_dict = dict()
    steps_days_dict = dict()
    steps_y_target_dict = dict()
   

    
    # concatenate feature_matrix_realised and feature_matrix_day and hol so that the lags have the time info
    # as well
#     feature_matrix_realised = np.concatenate((feature_matrix_realised,
#                                               feature_matrix_day,
#                                               feature_matrix_hol), axis=1)
#     print(f'feature_matrix_realised shape after concat is {feature_matrix_realised.shape}')
    
    # concatenate feature_matrix_day and feature_matrix_hol 
    days_matrix = np.concatenate((feature_matrix_day, feature_matrix_hol), axis=1)
    
    for step in time_steps_weather:
        steps_weather_gen_dict[f'{recognizer}_weather_gen_step_{step}'] = list()

    for step in time_steps_weather:
        steps_weather_load_dict[f'{recognizer}_weather_load_step_{step}'] = list()
        
    for step in time_steps_realised:
        steps_realised_dict[f'{recognizer}_realised_step_{step}'] = list()
    
    for step in time_steps_days:
        steps_days_dict [f'{recognizer}_days_step_{step}'] = list()
        
    for step in time_steps_days:
        steps_y_target_dict[f'{recognizer}_y_target_step_{step}'] = list()
        
        
    # iterate through each element of feature_matrix_fc_gen
    for i in range(len(target_matrix)):
        end_ix = i + max(max_steps_weather, max_steps_realised, max_steps_days)
        # check if we are beyond the dataset
        if end_ix+1 > len(target_matrix):
            break
        for step in time_steps_weather:
            steps_weather_gen_dict[f'{recognizer}_weather_gen_step_{step}'].append(feature_matrix_fc_gen[end_ix - step])

    # iterate through each element of feature_matrix_fc_load
    for i in range(len(target_matrix)):
        end_ix = i + max(max_steps_weather, max_steps_realised, max_steps_days)
        # check if we are beyond the dataset
        if end_ix+1 > len(target_matrix):
            break
        for step in time_steps_weather:
            steps_weather_load_dict[f'{recognizer}_weather_load_step_{step}'].append(feature_matrix_fc_load[end_ix - step])
    
    # iterate through each element of feature_matrix_realised
    for i in range(len(target_matrix)):
        end_ix = i + max(max_steps_weather, max_steps_realised, max_steps_days)
        # check if we are beyond the dataset
        if end_ix+1 > len(target_matrix):
            break
        for step in time_steps_realised:
            steps_realised_dict[f'{recognizer}_realised_step_{step}'].append(feature_matrix_realised[end_ix - step])
            
    for i in range(len(target_matrix)):
        end_ix = i + max(max_steps_weather, max_steps_realised, max_steps_days)
        # check if we are beyond the dataset
        if end_ix+1 > len(target_matrix):
            break
        for step in time_steps_days:
            steps_days_dict[f'{recognizer}_days_step_{step}'].append(days_matrix[end_ix - step])
            steps_y_target_dict[f'{recognizer}_y_target_step_{step}'].append(target_matrix[end_ix - step])
            
#         seq_y_target = target_matrix[end_ix]
#         seq_day = days_matrix[end_ix]
#         y_target.append(seq_y_target)
#         days.append(seq_day)
        
    # transform the lists in the dictionaries to numpy arrays     
    for key in steps_weather_gen_dict.keys():
        steps_weather_gen_dict[key] = np.array(steps_weather_gen_dict[key])
        
    for key in steps_weather_load_dict.keys():
        steps_weather_load_dict[key] = np.array(steps_weather_load_dict[key])
        
    for key in steps_realised_dict.keys():
        steps_realised_dict[key] = np.array(steps_realised_dict[key])
        
    for key in steps_days_dict.keys():
        steps_days_dict[key] = np.array(steps_days_dict[key])

    for key in steps_y_target_dict.keys():
        steps_y_target_dict[key] = np.array(steps_y_target_dict[key])

        

    return steps_weather_gen_dict, steps_weather_load_dict, steps_realised_dict, steps_days_dict, steps_y_target_dict

## Generate sequential features

In [6]:
time_steps_weather = [0]
time_steps_realised = np.arange(1,5,1).tolist()
time_steps_days = [0] #np.arange(1,97,1).tolist()

X_train_fc_gen, X_train_fc_load, X_train_realised, X_train_days, y_train = generator('train', 
                                                                                     time_steps_weather, 
                                                                                     time_steps_realised,
                                                                                     time_steps_days,
                                                                                     X_train_fc_gen, 
                                                                                     X_train_fc_load,
                                                                                     X_train_day,
                                                                                     X_train_hol,
                                                                                     X_train_realised,
                                                                                     y_train)


X_val_fc_gen, X_val_fc_load, X_val_realised, X_val_days, y_val = generator('val', 
                                                                             time_steps_weather, 
                                                                             time_steps_realised,
                                                                             time_steps_days, 
                                                                             X_val_fc_gen, 
                                                                             X_val_fc_load,
                                                                             X_val_day,
                                                                             X_val_hol,
                                                                             X_val_realised,
                                                                             y_val)


X_test_fc_gen, X_test_fc_load, X_test_realised, X_test_days, y_test = generator('test', 
                                                                                 time_steps_weather, 
                                                                                 time_steps_realised,
                                                                                 time_steps_days, 
                                                                                 X_test_fc_gen, 
                                                                                 X_test_fc_load,
                                                                                 X_test_day,
                                                                                 X_test_hol,
                                                                                 X_test_realised,
                                                                                 y_test)

In [7]:
# realised data has to be a one big matrix
X_train_realised = np.stack([var[1] for var in X_train_realised.items()])
X_train_realised = np.moveaxis(X_train_realised, 0, 1)

X_val_realised = np.stack([var[1] for var in X_val_realised.items()])
X_val_realised = np.moveaxis(X_val_realised, 0, 1)

X_test_realised = np.stack([var[1] for var in X_test_realised.items()])
X_test_realised = np.moveaxis(X_test_realised, 0, 1)

print(X_train_realised.shape, X_val_realised.shape, X_test_realised.shape)

# days data has to be a one big matrix
X_train_days = np.stack([var[1] for var in X_train_days.items()])
X_train_days = np.moveaxis(X_train_days, 0, 1)

X_val_days = np.stack([var[1] for var in X_val_days.items()])
X_val_days = np.moveaxis(X_val_days, 0, 1)

X_test_days = np.stack([var[1] for var in X_test_days.items()])
X_test_days = np.moveaxis(X_test_days, 0, 1)

print(X_train_days.shape, X_val_days.shape, X_test_days.shape)

# y data has to be a one big matrix
y_train = np.stack([var[1] for var in y_train.items()])
y_train = np.moveaxis(y_train, 0, 1)

y_val = np.stack([var[1] for var in y_val.items()])
y_val = np.moveaxis(y_val, 0, 1)

y_test = np.stack([var[1] for var in y_test.items()])
y_test = np.moveaxis(y_test, 0, 1)

print(y_train.shape, y_val.shape, y_test.shape)

(186712, 4, 4) (8636, 4, 4) (8636, 4, 4)
(186712, 1, 12) (8636, 1, 12) (8636, 1, 12)
(186712, 1, 4) (8636, 1, 4) (8636, 1, 4)


In [8]:
# transform each key/value of the feature_channel/weather to a global variable
for key,val in X_train_fc_gen.items():
    exec(key + '=val')
        
for key,val in X_val_fc_gen.items():
    exec(key + '=val')

for key,val in X_test_fc_gen.items():
    exec(key + '=val')
    

for key,val in X_train_fc_load.items():
    exec(key + '=val')
        
for key,val in X_val_fc_load.items():
    exec(key + '=val')

for key,val in X_test_fc_load.items():
    exec(key + '=val')

In [9]:
conv_part_train = train_weather_gen_step_0.astype('float32')
conv_part_val = val_weather_gen_step_0.astype('float32')
conv_part_test = test_weather_gen_step_0.astype('float32')

attn_part_train = X_train_realised.astype('float32')
attn_part_val = X_val_realised.astype('float32')
attn_part_test = X_test_realised.astype('float32')

tabular_part_train = X_train_days.astype('float32')
tabular_part_val = X_val_days.astype('float32')
tabular_part_test = X_test_days.astype('float32')

# target_train = y_train.reshape(-1,96*4).astype('float32')
# target_val = y_val.reshape(-1,96*4).astype('float32')
# target_test = y_test.astype('float32')


n_steps_conv = len(time_steps_weather)

n_steps = X_train_realised.shape[1]
n_features = X_train_realised.shape[2]

n_tab_in_steps = X_train_days.shape[1]
n_tab_in_features = X_train_days.shape[2]

#### Create dataframes for clarity

Weather

In [10]:
# ['temp', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'tsun'] ["Wind","Solar"]
#    0       1        2       3      4        5      6        7      8

The mean values are chosen for the weather data and the sum will be chosen for the installed capacity

In [11]:
df_weather_train = pd.DataFrame(np.mean(conv_part_train.reshape(conv_part_train.shape[0],-1,conv_part_train.shape[3]),axis=1))
df_weather_val = pd.DataFrame(np.mean(conv_part_val.reshape(conv_part_val.shape[0],-1,conv_part_val.shape[3]),axis=1))
df_weather_test = pd.DataFrame(np.mean(conv_part_test.reshape(conv_part_test.shape[0],-1,conv_part_test.shape[3]),axis=1))

In [12]:
df_weather_train.rename(columns={0 : 'temp',
                                 1 : 'rhum',
                                 2 : 'prcp',
                                 3 : 'wdir',
                                 4 : 'wspd',
                                 5 : 'pres',
                                 6 : 'tsun',
                                 7 : "Wind",
                                 8 : "Solar"},inplace=True)

df_weather_val.rename(columns={  0 : 'temp',
                                 1 : 'rhum',
                                 2 : 'prcp',
                                 3 : 'wdir',
                                 4 : 'wspd',
                                 5 : 'pres',
                                 6 : 'tsun',
                                 7 : "Wind",
                                 8 : "Solar"},inplace=True)

df_weather_test.rename(columns={ 0 : 'temp',
                                 1 : 'rhum',
                                 2 : 'prcp',
                                 3 : 'wdir',
                                 4 : 'wspd',
                                 5 : 'pres',
                                 6 : 'tsun',
                                 7 : "Wind",
                                 8 : "Solar"},inplace=True)

In [13]:
df_weather_train.loc[:,"Wind"] = (np.sum(conv_part_train.reshape(conv_part_train.shape[0],-1,conv_part_train.shape[3]),axis=1))[:,7]
df_weather_val.loc[:,"Wind"] = (np.sum(conv_part_val.reshape(conv_part_val.shape[0],-1,conv_part_val.shape[3]),axis=1))[:,7]
df_weather_test.loc[:,"Wind"] = (np.sum(conv_part_test.reshape(conv_part_test.shape[0],-1,conv_part_test.shape[3]),axis=1))[:,7]

df_weather_train.loc[:,"Solar"] = (np.sum(conv_part_train.reshape(conv_part_train.shape[0],-1,conv_part_train.shape[3]),axis=1))[:,8]
df_weather_val.loc[:,"Solar"] = (np.sum(conv_part_val.reshape(conv_part_val.shape[0],-1,conv_part_val.shape[3]),axis=1))[:,7]
df_weather_test.loc[:,"Solar"] = (np.sum(conv_part_test.reshape(conv_part_test.shape[0],-1,conv_part_test.shape[3]),axis=1))[:,8]

Realised

In [14]:
# ['Realised/System total load in MAW', 'Realised/Wind Offshore in MAW',
#        'Realised/Wind Onshore in MAW', 'Realised/Solar in MAW'] / ["1lag", "2lag" "etc."]

In [15]:
df_realised_train = pd.DataFrame(attn_part_train.reshape(attn_part_train.shape[0],-1))
df_realised_val = pd.DataFrame(attn_part_val.reshape(attn_part_val.shape[0],-1))
df_realised_test = pd.DataFrame(attn_part_test.reshape(attn_part_test.shape[0],-1))

In [16]:
for i in range(attn_part_train.shape[1]):
    df_realised_train.rename(columns={0+i*4 : 'Realised/System total load in MAW_'+ f'lag_{i+1}',
                                      1+i*4 : 'Realised/Wind Offshore in MAW_'+ f'lag_{i+1}',
                                      2+i*4 : 'Realised/Wind Onshore in MAW_'+ f'lag_{i+1}',
                                      3+i*4 : 'Realised/Solar in MAW_'+ f'lag_{i+1}'},inplace=True)
    
    df_realised_val.rename(columns={0+i*4 : 'Realised/System total load in MAW_'+ f'lag_{i+1}',
                                      1+i*4 : 'Realised/Wind Offshore in MAW_'+ f'lag_{i+1}',
                                      2+i*4 : 'Realised/Wind Onshore in MAW_'+ f'lag_{i+1}',
                                      3+i*4 : 'Realised/Solar in MAW_'+ f'lag_{i+1}'},inplace=True)
    
    df_realised_test.rename(columns={0+i*4 : 'Realised/System total load in MAW_'+ f'lag_{i+1}',
                                      1+i*4 : 'Realised/Wind Offshore in MAW_'+ f'lag_{i+1}',
                                      2+i*4 : 'Realised/Wind Onshore in MAW_'+ f'lag_{i+1}',
                                      3+i*4 : 'Realised/Solar in MAW_'+ f'lag_{i+1}'},inplace=True)

Entsoe + Date +  Holiday  

In [17]:
df_tabular_train = pd.DataFrame(data=tabular_part_train.reshape(-1,12),
                                 columns = ['Day ahead/System total load in MAW',
                                            'Day ahead/Solar in MAW',
                                            'Day ahead/Wind Onshore in MAW',
                                            'Day ahead/Wind Offshore in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday'])
df_tabular_val = pd.DataFrame(data=tabular_part_val.reshape(-1,12),
                               columns = ['Day ahead/System total load in MAW',
                                            'Day ahead/Solar in MAW',
                                            'Day ahead/Wind Onshore in MAW',
                                            'Day ahead/Wind Offshore in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday'])
df_tabular_test = pd.DataFrame(data=tabular_part_test.reshape(-1,12),
                                 columns = ['Day ahead/System total load in MAW',
                                            'Day ahead/Solar in MAW',
                                            'Day ahead/Wind Onshore in MAW',
                                            'Day ahead/Wind Offshore in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday'])

### Evaluation function

In [18]:
def MAE_MASE(y_true,y_pred):
    overall_mae = mean_absolute_error(y_true, y_pred)
    
    naive_forecast = y_true[1:]
    y_true_for_mase = y_true[:-1]
    mae_naive = mean_absolute_error(y_true_for_mase, naive_forecast)
    overall_mae_without_first_observation = mean_absolute_error(y_true[1:], y_pred[1:])

    overall_mase = overall_mae_without_first_observation/mae_naive
    
    return overall_mae, overall_mase

## Models 


Heard from a Kaggle Grandmaster:

Learning rate = 0.05, 1000 rounds, max depth = 3-5, subsample = 0.8-1.0, colsample_bytree = 0.3 - 0.8, lambda = 0 to 5

Add capacity to combat bias - add rounds

Reduce capacity to combat variance - depth / regularization


### Load

First iteration:
['temp', 'prcp', 'pres', 'tsun'] + [all lags load] + [load entsoe, all dates, holliday] 

In [19]:
df_load_w_train = df_weather_train.loc[:,['temp', 'prcp', 'pres', 'tsun']]
df_load_w_val = df_weather_val.loc[:,['temp', 'prcp', 'pres', 'tsun']]
df_load_w_test = df_weather_test.loc[:,['temp', 'prcp', 'pres', 'tsun']]

In [20]:
df_load_lag_train = df_realised_train.filter(regex='load')
df_load_lag_val = df_realised_val.filter(regex='load')
df_load_lag_test = df_realised_test.filter(regex='load')

In [21]:
df_load_tab_train = df_tabular_train.loc[:,['Day ahead/System total load in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday']]

df_load_tab_val = df_tabular_val.loc[:,['Day ahead/System total load in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday']]

df_load_tab_test = df_tabular_test.loc[:,['Day ahead/System total load in MAW',
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month',
                                            'sin_weekday', 
                                            'cos_weekday',
                                            'holliday']]


In [22]:
df_load_train = pd.concat([df_load_w_train, df_load_lag_train, df_load_tab_train], axis=1)
df_load_val = pd.concat([df_load_w_val, df_load_lag_val, df_load_tab_val], axis=1)
df_load_test = pd.concat([df_load_w_test, df_load_lag_test, df_load_tab_test], axis=1)

Target distribution in y_train is:

'Realised/System total load in MAW', 'Realised/Wind Offshore in MAW','Realised/Wind Onshore in MAW', 'Realised/Solar in MAW'

In [23]:
target_load_train = y_train.reshape(y_train.shape[0],-1)[:,0]
target_load_val = y_val.reshape(y_val.shape[0],-1)[:,0] 
target_load_test = y_test.reshape(y_test.shape[0],-1)[:,0]

In [24]:
load_xgb = xgbr(n_jobs=-1)
# load_xgb = xgbr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='XGB Load',
#                  n_jobs=-1, colsample_by_tree=0.8, reg_lambda=1#, random_state 
#                 )


load_brf = xgbrfr(n_jobs=-1)
# load_brf = xgbrfr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='BRF Load',
#                   n_jobs=-1, colsample_bytree=0.8, reg_lambda=1, #random_state
#                  )

In [25]:
load_xgb = load_xgb.fit(df_load_train,target_load_train)
load_brf = load_brf.fit(df_load_train,target_load_train)

/home/tjarke/anaconda3/envs/general/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[15:37:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:37:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [26]:
load_xgb_pred = load_xgb.predict(df_load_test)
load_brf_pred = load_brf.predict(df_load_test)

In [27]:
print("=================================")
print("Load XGB")
overall_mae, overall_mase = MAE_MASE(target_load_test,load_xgb_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")
print("=================================")
print("Load BRF")
overall_mae, overall_mase = MAE_MASE(target_load_test,load_brf_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")

Load XGB
The overall mean absolute error of the model in MW is: 371.11896874276283
The overall mean absolute error of the model in MW is: 0.7024875014235272
Load BRF
The overall mean absolute error of the model in MW is: 1161.304232916715
The overall mean absolute error of the model in MW is: 2.1979977268296884


### Generation Wind

First iteration:
['prcp', 'wdir', 'wspd', 'pres', 'Wind'] + [all lags wind gen] + [both wind entsoe, year, minute, month] 

In [28]:
df_wind_w_train = df_weather_train.loc[:,['prcp', 'wdir', 'wspd', 'pres', 'Wind']]
df_wind_w_val = df_weather_val.loc[:,['prcp', 'wdir', 'wspd', 'pres', 'Wind']]
df_wind_w_test = df_weather_test.loc[:,['prcp', 'wdir', 'wspd', 'pres', 'Wind']]

In [29]:
df_wind_lag_train = df_realised_train.filter(regex='Wind')
df_wind_lag_val = df_realised_val.filter(regex='Wind')
df_wind_lag_test = df_realised_test.filter(regex='Wind')

In [30]:
df_wind_tab_train = df_tabular_train.loc[:,['Day ahead/Wind Offshore in MAW',
                                            'Day ahead/Wind Onshore in MAW',                                            
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

df_wind_tab_val = df_tabular_val.loc[:,['Day ahead/Wind Offshore in MAW',
                                            'Day ahead/Wind Onshore in MAW',                                            
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

df_wind_tab_test = df_tabular_test.loc[:,['Day ahead/Wind Offshore in MAW',
                                            'Day ahead/Wind Onshore in MAW',                                            
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

In [31]:
df_wind_train = pd.concat([df_wind_w_train, df_wind_lag_train, df_wind_tab_train], axis=1)
df_wind_val = pd.concat([df_wind_w_val, df_wind_lag_val, df_wind_tab_val], axis=1)
df_wind_test = pd.concat([df_wind_w_test, df_wind_lag_test, df_wind_tab_test], axis=1)

In [32]:
target_wind_off_train = y_train.reshape(y_train.shape[0],-1)[:,1]
target_wind_off_val = y_val.reshape(y_val.shape[0],-1)[:,1] 
target_wind_off_test = y_test.reshape(y_test.shape[0],-1)[:,1]

target_wind_on_train = y_train.reshape(y_train.shape[0],-1)[:,2]
target_wind_on_val = y_val.reshape(y_val.shape[0],-1)[:,2] 
target_wind_on_test = y_test.reshape(y_test.shape[0],-1)[:,2]

In [33]:
wind_off_xgb = xgbr(n_jobs=-1)
# wind_off_xgb = xgbr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='XGB Wind_off',
#                  n_jobs=-1, colsample_by_tree=0.8, reg_lambda=1#, random_state 
#                 )

wind_off_brf = xgbrfr(n_jobs=-1)
# wind_off_brf = xgbrfr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='BRF Wind_off',
#                   n_jobs=-1, colsample_bytree=0.8, reg_lambda=1, #random_state
#                  )

wind_on_xgb = xgbr(n_jobs=-1)
# wind_on_xgb = xgbr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='XGB Wind_on',
#                  n_jobs=-1, colsample_by_tree=0.8, reg_lambda=1#, random_state 
#                 )

wind_on_brf = xgbrfr(n_jobs=-1)
# wind_on_brf = xgbrfr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='BRF Wind_on',
#                   n_jobs=-1, colsample_bytree=0.8, reg_lambda=1, #random_state
#                  )

In [34]:
wind_off_xgb = wind_off_xgb.fit(df_wind_train,target_wind_off_train)
wind_off_brf = wind_off_brf.fit(df_wind_train,target_wind_off_train)

wind_on_xgb = wind_on_xgb.fit(df_wind_train,target_wind_on_train)
wind_on_brf = wind_on_brf.fit(df_wind_train,target_wind_on_train)

/home/tjarke/anaconda3/envs/general/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[15:37:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:37:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:37:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:37:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [35]:
wind_off_xgb_pred = wind_off_xgb.predict(df_wind_test)
wind_off_brf_pred = wind_off_brf.predict(df_wind_test)

wind_on_xgb_pred = wind_on_xgb.predict(df_wind_test)
wind_on_brf_pred = wind_on_brf.predict(df_wind_test)

In [36]:
print("=================================")
print("Wind off XGB")
overall_mae, overall_mase = MAE_MASE(target_wind_off_test,wind_off_xgb_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")
print("=================================")
print("Wind off BRF")
overall_mae, overall_mase = MAE_MASE(target_wind_off_test,wind_off_brf_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")



print("=================================")
print("Wind on XGB")
overall_mae, overall_mase = MAE_MASE(target_wind_on_test,wind_on_xgb_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")
print("=================================")
print("Wind on BRF")
overall_mae, overall_mase = MAE_MASE(target_wind_on_test,wind_on_brf_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")

Wind off XGB
The overall mean absolute error of the model in MW is: 83.62393423383465
The overall mean absolute error of the model in MW is: 0.965558616239794
Wind off BRF
The overall mean absolute error of the model in MW is: 209.78118123473698
The overall mean absolute error of the model in MW is: 2.4221243631535665
Wind on XGB
The overall mean absolute error of the model in MW is: 185.0323199527912
The overall mean absolute error of the model in MW is: 0.9217270188444872
Wind on BRF
The overall mean absolute error of the model in MW is: 889.9912496392735
The overall mean absolute error of the model in MW is: 4.433066927901632


### Generation Solar

First iteration:
['temp', 'rhum', 'prcp', 'pres', 'tsun', 'Solar'] + [all lags solar gen] + [solar entsoe, year, minute, month]

In [37]:
df_solar_w_train = df_weather_train.loc[:,['temp', 'rhum', 'prcp', 'pres', 'tsun', 'Solar']]
df_solar_w_val = df_weather_val.loc[:,['temp', 'rhum', 'prcp', 'pres', 'tsun', 'Solar']]
df_solar_w_test = df_weather_test.loc[:,['temp', 'rhum', 'prcp', 'pres', 'tsun', 'Solar']]

In [38]:
df_solar_lag_train = df_realised_train.filter(regex='Solar')
df_solar_lag_val = df_realised_val.filter(regex='Solar')
df_solar_lag_test = df_realised_test.filter(regex='Solar')

In [39]:
df_solar_tab_train = df_tabular_train.loc[:,['Day ahead/Solar in MAW',                                            
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

df_solar_tab_val = df_tabular_val.loc[:,['Day ahead/Solar in MAW',                                            
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

df_solar_tab_test = df_tabular_test.loc[:,['Day ahead/Solar in MAW',                                           
                                            'Year',
                                            'sin_minute',
                                            'cos_minute',
                                            'sin_month',
                                            'cos_month']]

In [40]:
df_solar_train = pd.concat([df_solar_w_train, df_solar_lag_train, df_solar_tab_train], axis=1)
df_solar_val = pd.concat([df_solar_w_val, df_solar_lag_val, df_solar_tab_val], axis=1)
df_solar_test = pd.concat([df_solar_w_test, df_solar_lag_test, df_solar_tab_test], axis=1)

In [41]:
target_solar_train = y_train.reshape(y_train.shape[0],-1)[:,3]
target_solar_val = y_val.reshape(y_val.shape[0],-1)[:,3] 
target_solar_test = y_test.reshape(y_test.shape[0],-1)[:,3]

In [42]:
solar_xgb = xgbr(n_jobs=-1)
# solar_xgb = xgbr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='XGB Solar',
#                  n_jobs=-1, colsample_by_tree=0.8, reg_lambda=1#, random_state 
#                 )

solar_brf = xgbrfr(n_jobs=-1)
# solar_brf = xgbrfr(n_estimators=1000, max_depth=3, learning_rate=0.05, objectve='BRF Solar',
#                   n_jobs=-1, colsample_bytree=0.8, reg_lambda=1, #random_state
#                  )

In [43]:
solar_xgb = solar_xgb.fit(df_solar_train,target_solar_train)
solar_brf = solar_brf.fit(df_solar_train,target_solar_train)

[15:37:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/tjarke/anaconda3/envs/general/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[15:37:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [44]:
solar_xgb_pred = solar_xgb.predict(df_solar_test)
solar_brf_pred = solar_brf.predict(df_solar_test)

In [45]:
print("=================================")
print("Solar XGB")
overall_mae, overall_mase = MAE_MASE(target_solar_test,solar_xgb_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")
print("=================================")
print("Solar BRF")
overall_mae, overall_mase = MAE_MASE(target_solar_test,solar_brf_pred)
print(f"The overall mean absolute error of the model in MW is: {overall_mae}")
print(f"The overall mean absolute error of the model in MW is: {overall_mase}")
print("=================================")

Solar XGB
The overall mean absolute error of the model in MW is: 154.81323570013046
The overall mean absolute error of the model in MW is: 0.3789127384974852
Solar BRF
The overall mean absolute error of the model in MW is: 563.3137349530247
The overall mean absolute error of the model in MW is: 1.378724905969945
